# Analyzing AirBnB data for Washington DC
This project dives deep into the AirBnB data for Washington DC. Washington DC, as the capital of the United States, has recently been attracting a lot of re-development. In fact, Amazon announced recently that they will have the second head-quarter in Northen Virginia, which is just a several miles away into DC. Also, personally this is where I live and it makes the project more interesting for me.

## CRISP-DM
CRISP-DM (Cross Industry Standard Process for Data Mining) is the standard process to follow across various industries when mining and analyzing data. Specifically, CRISP-DM follows the below steps:
<b>
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Data Modeling
5. Evaluate the results
6. Delopyment
</b>
In this notebook, we are going to follow the process above for the analysis of AirBnB data for Washington DC.

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

### Business Understanding
The first part of the CRISP-DM is to understand the business problems. Key questions I would like to answer through this analysis are:
<b>
1. What are the factors affecting the rental price?
2. When is the most expensive/inexpensive to stay in DC?
3. How is the price volatility?
</b>
By understanding the first question, the hosts are able to take appropriate actions (e.g. investing in one of the amenities users value particularly) in order to maximize their revenue. Also, from AirBnB users perspective, understading the question 2) ad 3) help make judgement about their travel time.

### Data Understanding ad Data Preparation
The second and the third part of the CRISP-DM is data understanding and data preparation. Let's first read the data. We are going to use "listings.csv" data to understand the factors affecting the price. To understand the seasonal trend and volatility, we are going to use "calendar.csv" as it contains the price history.

In [102]:
dc_listings = pd.read_csv('.\\data\\DC\\listings.csv')
dc_calendar = pd.read_csv('.\\data\\DC\\calendar.csv',index_col='date', parse_dates=True)

C:\Users\owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Listings Data
We are going to start with Listings Data

In [5]:
dc_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,3362,https://www.airbnb.com/rooms/3362,20181115100708,2018-11-15,"Convention Center Rowhouse & In Law: 2 Units, 4BR",An architect-designed rowhouse featuring a lar...,This listing features our 19th century Victori...,An architect-designed rowhouse featuring a lar...,none,NaN,...,f,NaN,"{""DISTRICT OF COLUMBIA"","" WASHINGTON""}",t,f,strict_14_with_grace_period,f,f,5,1.27
1,3662,https://www.airbnb.com/rooms/3662,20181115100708,2018-11-15,Vita's Hideaway II,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,NaN,"{""DISTRICT OF COLUMBIA"","" WASHINGTON""}",f,f,moderate,f,f,3,0.38
2,3686,https://www.airbnb.com/rooms/3686,20181115100708,2018-11-15,Vita's Hideaway,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,NaN,"{""DISTRICT OF COLUMBIA"","" WASHINGTON""}",f,f,moderate,f,f,3,0.69
3,3771,https://www.airbnb.com/rooms/3771,20181115100708,2018-11-15,Mt. Pleasant,"Clean, convenient and welcoming home. Bedroom...","Spacious, clean, convenient and welcoming home...","Clean, convenient and welcoming home. Bedroom...",none,NaN,...,f,NaN,"{""DISTRICT OF COLUMBIA"","" WASHINGTON""}",f,f,moderate,f,f,1,0.01
4,4002,https://www.airbnb.com/rooms/4002,20181115100708,2018-11-15,2 Bedroom Private Condo Suite (Basement Apt),This is a very large beautiful English BASEMEN...,This is a very large (1400 SQ feet) beautiful ...,This is a very large beautiful English BASEMEN...,none,This is a very quite neighborhood.,...,f,NaN,"{""DISTRICT OF COLUMBIA"","" WASHINGTON""}",t,f,flexible,f,t,1,1.25


In [6]:
dc_listings.isnull().sum()

id                                     0
listing_url                            0
scrape_id                              0
last_scraped                           0
name                                   4
summary                              224
space                               2608
description                           73
experiences_offered                    0
neighborhood_overview               3112
notes                               4942
transit                             2836
access                              3159
interaction                         3406
house_rules                         3430
thumbnail_url                       9369
medium_url                          9369
picture_url                            0
xl_picture_url                      9369
host_id                                0
host_url                               0
host_name                              1
host_since                             1
host_location                         33
host_about      

You can see that there are many columns which have many missing data points. Let's remove columns with more than 100 data points missing.

In [7]:
dc_listings = dc_listings[dc_listings.columns[dc_listings.isnull().sum()<100]]
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9369 entries, 0 to 9368
Data columns (total 61 columns):
id                                  9369 non-null int64
listing_url                         9369 non-null object
scrape_id                           9369 non-null int64
last_scraped                        9369 non-null object
name                                9365 non-null object
description                         9296 non-null object
experiences_offered                 9369 non-null object
picture_url                         9369 non-null object
host_id                             9369 non-null int64
host_url                            9369 non-null object
host_name                           9368 non-null object
host_since                          9368 non-null object
host_location                       9336 non-null object
host_is_superhost                   9368 non-null object
host_thumbnail_url                  9368 non-null object
host_picture_url                    9368 

Now we see that most of the columns have valid data entries. Additionally, let's drop the following columns as these columns should not have any implication in terms of price.

In [8]:
drop_cols = ['listing_url',
            'scrape_id',
             'name',
             'picture_url',
             'host_location',
             'host_id',
             'host_name',
             'host_url',
             'host_thumbnail_url',
             'host_picture_url',
            ]

In [9]:
dc_listings = dc_listings.drop(drop_cols, axis=1)
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9369 entries, 0 to 9368
Data columns (total 51 columns):
id                                  9369 non-null int64
last_scraped                        9369 non-null object
description                         9296 non-null object
experiences_offered                 9369 non-null object
host_since                          9368 non-null object
host_is_superhost                   9368 non-null object
host_listings_count                 9368 non-null float64
host_total_listings_count           9368 non-null float64
host_verifications                  9369 non-null object
host_has_profile_pic                9368 non-null object
host_identity_verified              9368 non-null object
street                              9369 non-null object
neighbourhood_cleansed              9369 non-null object
city                                9367 non-null object
state                               9358 non-null object
market                              9

Additionally, since the location information is already contained in the latitude and the longitude, we can drop the following columns.

In [10]:
drop_loc_cols = ['street',
                'city',
                'state',
                'country_code',
                'country',
                'jurisdiction_names']

In [11]:
dc_listings = dc_listings.drop(drop_loc_cols, axis=1)
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9369 entries, 0 to 9368
Data columns (total 45 columns):
id                                  9369 non-null int64
last_scraped                        9369 non-null object
description                         9296 non-null object
experiences_offered                 9369 non-null object
host_since                          9368 non-null object
host_is_superhost                   9368 non-null object
host_listings_count                 9368 non-null float64
host_total_listings_count           9368 non-null float64
host_verifications                  9369 non-null object
host_has_profile_pic                9368 non-null object
host_identity_verified              9368 non-null object
neighbourhood_cleansed              9369 non-null object
market                              9340 non-null object
smart_location                      9369 non-null object
latitude                            9369 non-null float64
longitude                           

Some of the features here are self-explanatory, but some are not. In particular, 'object' type 

In [12]:
dc_listings['market'].value_counts()

D.C.                     9336
Santiago                    1
Other (International)       1
Dubai                       1
Toms River                  1
Name: market, dtype: int64

In [13]:
dc_listings['experiences_offered'].value_counts()

none    9369
Name: experiences_offered, dtype: int64

In [14]:
dc_listings['host_verifications'].value_counts()

['email', 'phone', 'reviews', 'kba']                                                                                                                              1254
['email', 'phone', 'reviews']                                                                                                                                      882
['email', 'phone', 'reviews', 'jumio', 'government_id']                                                                                                            700
['email', 'phone']                                                                                                                                                 674
['email', 'phone', 'reviews', 'jumio', 'offline_government_id', 'government_id']                                                                                   465
['email', 'phone', 'reviews', 'jumio', 'offline_government_id', 'selfie', 'government_id', 'identity_manual']                                                      40

We can drop these features
'market': Almost all the entries are D.C. as it's supposed to be.
'experiences_offered': All the entries are 'none'
'host_verifications': It looks like how the host is verified. But intuitively how the host is verified should not affect the listing price.

In [16]:
drop_cols = ['experiences_offered',
                 'host_verifications',
                 'market']
dc_listings = dc_listings.drop(drop_cols, axis=1)
dc_listings.isnull().sum()

id                                   0
last_scraped                         0
description                         73
host_since                           1
host_is_superhost                    1
host_listings_count                  1
host_total_listings_count            1
host_has_profile_pic                 1
host_identity_verified               1
neighbourhood_cleansed               0
smart_location                       0
latitude                             0
longitude                            0
is_location_exact                    0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            9
bedrooms                             8
beds                                12
bed_type                             0
amenities                            0
price                                0
guests_included                      0
extra_people                         0
minimum_nights           

There seems to be record which has null in host information (host_since, host_is_super_host, host_listings_count etc). Let's remove this record.

In [17]:
dc_listings = dc_listings[pd.notnull(dc_listings['host_since'])]
dc_listings.isnull().sum()

id                                   0
last_scraped                         0
description                         73
host_since                           0
host_is_superhost                    0
host_listings_count                  0
host_total_listings_count            0
host_has_profile_pic                 0
host_identity_verified               0
neighbourhood_cleansed               0
smart_location                       0
latitude                             0
longitude                            0
is_location_exact                    0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            9
bedrooms                             8
beds                                12
bed_type                             0
amenities                            0
price                                0
guests_included                      0
extra_people                         0
minimum_nights           

There are still 3 features which has missing values. These are numerical features, and they can be filled with median value.

In [18]:
dc_listings['beds'] = dc_listings['beds'].fillna(dc_listings['beds'].median())
dc_listings['bathrooms'] = dc_listings['bathrooms'].fillna(dc_listings['bathrooms'].median())
dc_listings['bedrooms'] = dc_listings['bedrooms'].fillna(dc_listings['bedrooms'].median())
dc_listings.isnull().sum()

id                                   0
last_scraped                         0
description                         73
host_since                           0
host_is_superhost                    0
host_listings_count                  0
host_total_listings_count            0
host_has_profile_pic                 0
host_identity_verified               0
neighbourhood_cleansed               0
smart_location                       0
latitude                             0
longitude                            0
is_location_exact                    0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            0
bedrooms                             0
beds                                 0
bed_type                             0
amenities                            0
price                                0
guests_included                      0
extra_people                         0
minimum_nights           

Intuitively, what amenities the place has is an important considering when deciding on the accommodation, and it should have some impact on the price.

In [19]:
dc_listings['amenities'].head(10)

0    {TV,Internet,Wifi,"Air conditioning",Kitchen,"...
1    {TV,Internet,Wifi,Kitchen,"Free parking on pre...
2    {Internet,Wifi,Kitchen,"Free street parking","...
3    {"Cable TV","Air conditioning",Heating,"Smoke ...
4    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
5    {TV,Wifi,"Air conditioning",Kitchen,"Pets live...
6    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
7    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
8    {Internet,Wifi,"Air conditioning",Kitchen,"Fre...
9    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
Name: amenities, dtype: object

In [20]:
dc_listings['amenities'].str.strip('{}').str.replace('"','').str.split(',').str.join('|').str.get_dummies()

,toilet,24-hour check-in,Accessible-height bed,Accessible-height toilet,Air conditioning,Amazon Echo,BBQ grill,Baby bath,Baby monitor,Babysitter recommendations,...,Wide clearance to bed,Wide clearance to shower,Wide doorway,Wide entryway,Wide hallway clearance,Wifi,Window guards,Wine cooler,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
6,0,1,0,0,1,0,1,0,0,1,...,1,0,1,1,1,1,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


Convert 'price' and 'extra_people' to  the numerical value 

In [221]:
dc_listings['price'].str.replace('$','').str.replace(',','').astype(float)
dc_listings['extra_people'] = dc_listings['extra_people'].str.replace('$','').astype(float)

In [223]:
dc_listings['calendar_updated'].unique()
dc_listings['calendar_updated'].str.contains('weeks ago')
dc_listings.loc[dc_listings['calendar_updated'].str.contains('days ago'), 'calendar_updated'] = dc_listings.loc[dc_listings['calendar_updated'].str.contains('days ago'), 'calendar_updated'].str.replace(' days ago', '').astype(float)*1
dc_listings['calendar_updated'].str.contains('weeks ago')
#dc_listings.loc[dc_listings['calendar_updated'].str.contains('weeks ago'), 'calendar_updated']
#dc_listings.loc[dc_listings['calendar_updated'].str.contains('months ago'), 'calendar_updated'] = dc_listings.loc[dc_listings['calendar_updated'].str.contains('months ago'), 'calendar_updated'].str.replace(' months ago', '').astype(float)*30

0         NaN
1        True
2        True
3       False
4       False
5        True
6       False
7       False
8       False
9       False
10      False
11       True
12       True
13      False
14       True
15        NaN
16      False
17       True
18      False
19      False
20      False
21      False
22        NaN
23       True
24        NaN
25      False
26      False
27      False
28      False
29      False
        ...  
9339      NaN
9340      NaN
9341      NaN
9342    False
9343      NaN
9344      NaN
9345      NaN
9346      NaN
9347      NaN
9348    False
9349    False
9350    False
9351    False
9352    False
9353    False
9354    False
9355    False
9356    False
9357    False
9358    False
9359    False
9360    False
9361    False
9362    False
9363    False
9364    False
9365    False
9366    False
9367    False
9368    False
Name: calendar_updated, Length: 9368, dtype: object

In [226]:
dc_listings['calendar_updated'].head(20)

0                 2
1       3 weeks ago
2       3 weeks ago
3     28 months ago
4         yesterday
5       3 weeks ago
6        1 week ago
7     18 months ago
8      2 months ago
9             today
10     2 months ago
11      2 weeks ago
12      3 weeks ago
13        yesterday
14      3 weeks ago
15                6
16     2 months ago
17      3 weeks ago
18     2 months ago
19            today
Name: calendar_updated, dtype: object

In [146]:
dc_listings['calendar_updated'].head(10)

0     2 days ago
1    3 weeks ago
2    3 weeks ago
3            840
4      yesterday
5    3 weeks ago
6     1 week ago
7            540
8             60
9          today
Name: calendar_updated, dtype: object

In [119]:
rpl_list = ['days ago', 'weeks ago', 'months ago', ]
for rpl in rpl_list:
    dc_listings['calendar_updated'] = dc_listings['calendar_updated'].str.replace(rpl,'')
dc_listings['calendar_updated'].unique()

array(['2 ', '3 ', '28 ', 'yesterday', '1 week ago', '18 ', 'today', '6 ',
       '4 ', '5 ', '8 ', '13 ', '7 ', '11 ', '10 ', '9 ', 'a week ago',
       '34 ', '81 ', '12 ', '32 ', '25 ', '22 ', 'never', '30 ', '35 ',
       '56 ', '14 ', '19 ', '31 ', '23 ', '17 ', '21 ', '24 ', '33 ',
       '29 ', '27 ', '15 ', '16 ', '45 ', '48 ', '49 ', '38 ', '36 ',
       '41 ', '47 ', '20 ', '46 ', '39 ', '26 ', '52 ', '42 ', '44 ',
       '37 ', '43 ', '40 '], dtype=object)

In [ ]:
{'today':0, 'yesterday':1, 'never':1000}

#### Calendar Data
Let's now look at the calendar data

In [103]:
dc_calendar.head(10)

,listing_id,available,price
date,,,
2019-09-18,3362,t,$433.00
2019-09-17,3362,t,$433.00
2019-09-16,3362,t,$433.00
2019-09-15,3362,t,$433.00
2019-09-14,3362,t,$450.00
2019-09-13,3362,t,$450.00
2019-09-12,3362,t,$433.00
2019-09-11,3362,t,$433.00
2019-09-10,3362,t,$433.00


In [104]:
dc_calendar.isnull().sum()

listing_id          0
available           0
price         2114655
dtype: int64

There are many null entries for price. This is because when the property is not avaialbe for rent (i.e. available column ='f'), the price is null. We can these rows. Also, since whenever there's price, it's always available, so we do not need 'available' column. The price column is string with $ sign again (same as listing data), so we convert it to numerical value.

In [105]:
dc_calendar = dc_calendar.dropna(axis=0)
dc_calendar = dc_calendar.drop(['available'],axis=1)
dc_calendar['price'] = dc_calendar['price'].str.replace('$','').str.replace(',','').astype(float)
dc_calendar.head()

,listing_id,price
date,,
2019-09-18,3362,433.0
2019-09-17,3362,433.0
2019-09-16,3362,433.0
2019-09-15,3362,433.0
2019-09-14,3362,450.0


Now we are interested in how the average price varies between the dates. We can take use groupby function of pandas, and group by 'date' column to take the average. At the same time, we are not interested in particular listing, so we can drop the listing_id columns.

In [106]:
dc_calendar_date = dc_calendar.groupby('date').mean()
dc_calendar_date.drop('listing_id',axis=1, inplace=True)
dc_calendar_date.head()

,price
date,
2018-11-15,245.773449
2018-11-16,239.419498
2018-11-17,239.675428
2018-11-18,214.371325
2018-11-19,207.908384


Now if we look at the value counts below, some dates only have 1 listing. It does not make sense to take an average of 1 listing, so we will drop those dates.

In [111]:
dc_calendar.groupby('date').count()

,listing_id,price
date,,
2018-11-15,1386,1386
2018-11-16,2031,2031
2018-11-17,2397,2397
2018-11-18,3299,3299
2018-11-19,3471,3471
2018-11-20,3313,3313
2018-11-21,2931,2931
2018-11-22,2858,2858
2018-11-23,2947,2947


In [112]:
dc_calendar_date = dc_calendar_date.drop(pd.date_range('2019-11-15', '2019-11-20'), errors='ignore')

In [113]:
dc_calendar_date['price'].describe()

count    365.000000
mean     210.271789
std       11.499001
min      180.367550
25%      202.383122
50%      210.667733
75%      217.683515
max      247.709002
Name: price, dtype: float64